# Advanced Lane Finding

## Importing necessary packages

In [4]:
#importing some useful packages required for this project
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pickle
import numpy as np
import cv2
import math
import os
%matplotlib inline

## Camera Calibration

In [ ]:
cam_cal_out = 'camera_cal_output'
objpoints = [] # 3d points in real world
imgpoints = [] # 2d points on a plane

# Make object points according to the calibration images
# Found images with the corners: 9x4, 9x5, 9x6 , 8x6, 7x6, 5x6
# 9x4
objpts9x4 = np.zeros((4*9, 3), np.float32)
objpts9x4[:,:2] = np.mgrid[0:9,0:4].T.reshape(-1,2)

# 9x5
objpts9x5 = np.zeros((5*9, 3), np.float32)
objpts9x5[:,:2] = np.mgrid[0:9,0:5].T.reshape(-1,2)

# 9x6
objpts9x6 = np.zeros((6*9, 3), np.float32)
objpts9x6[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# 8x6
objpts8x6 = np.zeros((6*8, 3), np.float32)
objpts8x6[:,:2] = np.mgrid[0:8,0:6].T.reshape(-1,2)

# 7x6
objpts7x6 = np.zeros((6*7, 3), np.float32)
objpts7x6[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)

# 5x6
objpts5x6 = np.zeros((6*5, 3), np.float32)
objpts5x6[:,:2] = np.mgrid[0:5,0:6].T.reshape(-1,2)

# Getting the images from the camera_cal directory
img_names = os.listdir('camera_cal/')
for img_name in img_names:
    if img_name.endswith('.jpg'):
        
        #Preparing for subplots of different Processes
        plt.figure(figsize=(14,12))
        
        # Reading the image file using cv2
        img = mpimg.imread('camera_cal/' + img_name)
        
        # Subplot for the original image
        plt.subplot(121)
        plt.title("Original  Image: "+ img_name)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(img)
        
        # Making a copy for displaying the processed image later on. We do not want to disrupt the original one
        img_processed = np.copy(img)
        img_size = (img.shape[1], img.shape[0])
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # BGR since the image was read using cv2.imread

        # Checking for the appropriate corners and storing it
        ret, corners = cv2.findChessboardCorners(gray, (9,4), None)
        objpts = objpts9x4
        if not ret:
            ret, corners = cv2.findChessboardCorners(gray, (9,5), None)
            objpts = objpts9x5
        if not ret:
            ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
            objpts = objpts9x6
        if not ret:
            ret, corners = cv2.findChessboardCorners(gray, (8,6), None)
            objpts = objpts8x6
        if not ret:
            ret, corners = cv2.findChessboardCorners(gray, (7,6), None)
            objpts = objpts7x6
        if not ret:
            ret, corners = cv2.findChessboardCorners(gray, (5,6), None)
            objpts = objpts5x6

        # Draw the chessboard corners
        if ret == True:
            cv2.drawChessboardCorners(img_processed,(corners.shape[1],corners.shape[0]), corners, ret)

            # Preparing to calibrate the camera
            objpoints.append(objpts)
            imgpoints.append(corners)

            # Storing the required calibration data in the global variables
            ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)
            img_processed = cv2.undistort(img_processed, mtx, dist, None, mtx)
            
        # Subplot for the processed image
        plt.subplot(122)
        plt.title("Corrected  Image: "+ img_name)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(img_processed)

        plt.show()
        
        if not os.path.exists(cam_cal_out):
            os.mkdir(cam_cal_out)
        cv2.imwrite(os.path.join(cam_cal_out , img_name), img_processed)

## Helper Functions For Test Image Pipeline

### Helper functions for  Reading Image, Conversion to grayscale

In [ ]:
# Reading images from the test directory
def read_image(img):
    testImageDir = 'test_images'
    return plt.imread(testImageDir + '/' + img)

# Converting an image to grayscale
def grayscale(img):
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)


### Helper functions for Image undistortion

In [ ]:
# Undistorting the image
def undistort_frame(img):
    return cv2.undistort(img, mtx, dist, None, mtx)

### Helper functions for Binary Threshold

In [ ]:
  def combined_threshold(img):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    s_thresh_min = 150
    s_thresh_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    thresh_min = 20
    thresh_max = 100
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    return combined_binary


### Helper Function for Perspective Transformation

In [ ]:
def perspective_transform(img):
    # Define 4 source and 4 destination points = np.float32([[,],[,],[,],[,]])
    src = np.float32([
        [568, 468.5],
        [715, 468.5],
        [1041.5, 679.1],
        [269.5, 679.1]
    ])
    
    dest = np.float32([
        [400, 0],
        [880, 0],
        [880, 710],
        [400, 710]
    ])
    
    M = cv2.getPerspectiveTransform(src, dest)
    Minv = cv2.getPerspectiveTransform(dest, src)
    
    warped_img = cv2.warpPerspective(img, M, (1280, 720), flags=cv2.INTER_LINEAR)
    return warped_img, M, Minv

### Helper functions for Fitting Polynomial

In [ ]:
def find_lane_pixels(binary_warped):
    # Take a histogram along all the columns in the lower half of the image.
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:, :], axis=0)

    # Find x-positions of the base of the lanes using the two most prominent peaks in the histogram.
    mid = np.int(len(histogram)//2)
    left_win_center = np.argmax(histogram[0:mid])
    right_win_center = np.argmax(histogram[mid:]) + mid
    
    # Use a sliding window to find and follow the lines up to the top of the frame
    nwindows = 9
    window_height = np.int(binary_warped.shape[0]/nwindows)


    margin = 100
    nonzero = binary_warped.nonzero()
    min_nonzero_pixel_count = 200
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    left_lane_inds = []
    right_lane_inds = []
    # print("nonzero len {}".format(len(nonzero[0])))
    # out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

    h = binary_warped.shape[0]
    left_windows = []
    right_windows = []
    for i in range(nwindows):
        # Create two rectangular windows
        win_top = h - (i + 1) * window_height
        win_bottom = win_top + window_height
        left_win_left = left_win_center - margin
        left_win_right = left_win_center + margin
        right_win_left = right_win_center - margin
        right_win_right = right_win_center + margin
        # print("top {}, bottom {}, left win ({}, {}), right win ({} {})".format(win_top, win_bottom, left_win_left, left_win_right, right_win_left, right_win_right))
        
        # cv2.rectangle(out_img, (left_win_left, win_bottom), (left_win_right, win_top), (0, 255, 0), 2)
        # cv2.rectangle(out_img, (right_win_left, win_bottom), (right_win_right, win_top), (0, 255, 0), 2)
        left_windows.append(((left_win_left, win_bottom), (left_win_right, win_top)))
        right_windows.append(((right_win_left, win_bottom), (right_win_right, win_top)))
        
        # Identify the nonzero pixels in x and y within the window
        left_inds = ((nonzeroy >= win_top) & (nonzeroy < win_bottom) & (nonzerox >= left_win_left) & (nonzerox < left_win_right)).nonzero()[0]
        right_inds = ((nonzeroy >= win_top) & (nonzeroy < win_bottom) & (nonzerox >= right_win_left) & (nonzerox < right_win_right)).nonzero()[0]
        # print("left pixel count {}, right pixel count {}".format(len(left_inds), len(right_inds)))
        # Append these indices to the lists
        left_lane_inds.append(left_inds)
        right_lane_inds.append(right_inds)
        
        # Move the center of windows
        if len(left_inds) > min_nonzero_pixel_count:
            left_win_center = np.int(np.mean(nonzerox[left_inds]))
        if len(right_inds) > min_nonzero_pixel_count:
            right_win_center = np.int(np.mean(nonzerox[right_inds]))

    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    return leftx, lefty, rightx, righty, left_windows, right_windows
    
def find_lane(binary_warped):
    leftx, lefty, rightx, righty, left_windows, right_windows = find_lane_pixels(binary_warped)

    if (len(leftx) == 0):
        left_fit = None
    else:
        left_fit = np.polyfit(lefty, leftx, 2)
    if (len(rightx) == 0):
        right_fit = None
    else:
        right_fit = np.polyfit(righty, rightx, 2)
    
    return left_fit, right_fit

def get_fit_lines(warped, left_fit, right_fit):
    ploty = np.linspace(0, warped.shape[0]-1, img.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    return left_fitx, right_fitx, ploty

def find_lane_fast(binary_warped, previuos_left_fit, previous_right_fit):
    
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_center = previuos_left_fit[0]*(nonzeroy**2) + previuos_left_fit[1]*nonzeroy + previuos_left_fit[2]
    right_lane_center = previous_right_fit[0]*(nonzeroy**2) + previous_right_fit[1]*nonzeroy + previous_right_fit[2]
    left_lane_inds = ((nonzerox > (left_lane_center - margin)) & (nonzerox < (left_lane_center + margin))) 
    right_lane_inds = ((nonzerox > (right_lane_center - margin)) & (nonzerox < (right_lane_center + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    return left_fit, right_fit


### Calculating Radius of Curvature

In [ ]:
def calculate_curvature(fitx, ploty):
    """ Calculate radius
    Args:
        fitx: x values of fitted line in pixel space
        ploty: y values of fitted line in pixel space
        
    Returns: raduis in meters at the bottom of image
    """
    x_m_per_pix = 3.7/480
    y_m_per_pix = 10/130.0
    
    fitx_m = fitx * x_m_per_pix
    ploty_m = ploty * y_m_per_pix

    y_m = np.max(ploty_m)
    fit = np.polyfit(ploty_m, fitx_m, 2)
    curverad = ((1 + (2 * fit[0] * y_m + fit[1])**2)**1.5) / np.absolute(2 * fit[0])
    return curverad

def calculate_average_curvature(binary_warped, left_fit, right_fit):
    leftx, rightx, ploty = get_fit_lines(binary_warped, left_fit, right_fit)
    left_radius = calculate_curvature(leftx, ploty)
    right_radius = calculate_curvature(rightx, ploty)
    return (left_radius + right_radius) / 2

### Calculating Distance from center

In [ ]:
def get_lane_values(left_fitx, right_fitx, ploty):
    x_m_per_pix = 3.7/480
    
    mean_distance = np.mean(right_fitx - left_fitx) * x_m_per_pix
    
    lane_center = (left_fitx[-1] + right_fitx[-1]) // 2
    car_center = 1280/2
    center_offset = (lane_center - car_center) * x_m_per_pix
    
    return mean_distance, center_offset

def check_sanity(binary_warped, left_fit, right_fit):
    leftx, rightx, ploty = get_fit_lines(binary_warped, left_fit, right_fit)    
    mean_distance, center_offset = get_lane_values(leftx, rightx, ploty)
    
    if mean_distance < 2.5 or mean_distance > 5:
        return False
    if center_offset > 2:
        return False
    
    left_radius = calculate_curvature(left_fitx, ploty)
    right_radius = calculate_curvature(right_fitx, ploty)
    radius = (left_radius + right_radius)/2
    if radius < 30:
        return False
    
    if radius < 10000 and (left_fit[0] * right_fit[0]) < 0:
        return False
    return True

### Drawing Line and Text

In [ ]:
def draw_lane(undist, binary_warped, left_fit, right_fit, Minv):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    left_fitx, right_fitx, ploty = get_fit_lines(binary_warped, left_fit, right_fit)

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undist.shape[1], undist.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    return result

def draw_text(img, radius, center_offset):
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(img,"radius {:.2f}m, center_offset {:.2f}m".format(radius, center_offset),(50,50), font, 1,(0,255,0),2)
    return img


## Reading the test Images from the directory

In [ ]:
image_files = os.listdir('test_images')

## Pipeline for Test Images

In [ ]:
dist_ouput_dir = 'output_images/distortion_corrected' # For storing the undistorted test images

thresh_ouput_dir = 'output_images/binary_image' # For storing the binary threshold result on test images

perspective_ouput_dir = 'output_images/perspective_transformed' # For storing transformation result on test images

polynomial_ouput_dir = 'output_images/polynomial_fit' # For storing polyfit on test images

final_ouput_dir = 'output_images/final_image' # For storing polyfit on test images

for image_file in image_files:
    if image_file.endswith('.jpg'):
        image = read_image(image_file)
        
        #Preparing for subplots of different Processes
        plt.figure(figsize=(14,12))
        
        plt.subplot(321)
        plt.title("Original Image: "+ image_file)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(image)
        
        # Undistort test images
        undistort_img = undistort_frame(image)
        plt.subplot(322)
        plt.title("Undistorted Image: "+ image_file)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(undistort_img)
        if not os.path.exists(dist_ouput_dir):
            os.mkdir(dist_ouput_dir)
        cv2.imwrite(os.path.join(dist_ouput_dir , image_file), undistort_img)
        
        # Convert the image in grayscale
        binary_image = combined_threshold(undistort_img)
        plt.subplot(323)
        plt.title("Threshold Image: "+ image_file)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(binary_image, cmap='gray')
        if not os.path.exists(thresh_ouput_dir):
            os.mkdir(thresh_ouput_dir)
        cv2.imwrite(os.path.join(thresh_ouput_dir , image_file), binary_image)
        
        # Convert the image in grayscale
        perspective_img, M, Minv = perspective_transform(binary_image)
        plt.subplot(324)
        plt.title("Perspective Image: "+ image_file)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(perspective_img, cmap='gray')
        if not os.path.exists(perspective_ouput_dir):
            os.mkdir(perspective_ouput_dir)
        cv2.imwrite(os.path.join(perspective_ouput_dir , image_file), perspective_img)
        
        # Convert the image in grayscale
        leftx, lefty, rightx, righty, left_windows, right_windows = find_lane_pixels(perspective_img)
        left_fit, right_fit = find_lane(perspective_img)
        left_fitx, right_fitx, ploty = get_fit_lines(perspective_img, left_fit, right_fit)
        polynomial_image = np.dstack((perspective_img, perspective_img, perspective_img))*255
        for win in left_windows:
            cv2.rectangle(polynomial_image, win[0], win[1], (0, 255, 0), 2)
        for win in right_windows:
            cv2.rectangle(polynomial_image, win[0], win[1], (0, 255, 0), 2)
            
        plt.subplot(325)
        plt.title("Polynomial Fit: "+ image_file)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(polynomial_image)
        plt.plot(left_fitx, ploty, color='yellow')
        plt.plot(right_fitx, ploty, color='yellow')
        plt.xlim(0, perspective_img.shape[1])
        plt.ylim(perspective_img.shape[0], 0)
        
        if not os.path.exists(polynomial_ouput_dir):
            os.mkdir(polynomial_ouput_dir)
        cv2.imwrite(os.path.join(polynomial_ouput_dir , image_file), polynomial_image)
        
        left_fit, right_fit = find_lane(perspective_img)
        final_img = draw_lane(undistort_img, perspective_img, left_fit, right_fit, Minv)
        leftx, rightx, ploty = get_fit_lines(perspective_img, left_fit, right_fit)
        mean_distance, center_offset = get_lane_values(leftx, rightx, ploty)
        radius = calculate_average_curvature(perspective_img, left_fit, right_fit)
        final_img = draw_text(final_img, radius, center_offset)
        plt.subplot(326)
        plt.title("Final Image: "+ image_file)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(final_img)
        if not os.path.exists(final_ouput_dir):
            os.mkdir(final_ouput_dir)
        cv2.imwrite(os.path.join(final_ouput_dir , image_file), final_img)
        
        plt.show()
        print("")


## Pipeline for Video Processing

In [ ]:
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        
Left=Line()
Right=Line()
no_of_bad_lines=13

def process_frame(img):
    
    undistorted = undistort_frame(img)
    masked = combined_threshold(undistorted)
    binary_warped, M, Minv = perspective_transform(masked)
    left_fit = None
    right_fit = None
    if Left.best_fit == None or Right.best_fit == None:
        left_fit, right_fit = find_lane(binary_warped)
    else:
        left_fit, right_fit = find_lane_fast(binary_warped, Left.best_fit, Right.best_fit)
        if not check_sanity(binary_warped, left_fit, right_fit):
            left_fit, right_fit = find_lane(binary_warped)
    Left.best_fit = left_fit
    Right.best_fit = right_fit
    leftx, rightx, ploty = get_fit_lines(binary_warped, left_fit, right_fit)
    mean_distance, center_offset = get_lane_values(leftx, rightx, ploty)
    radius = calculate_average_curvature(binary_warped, left_fit, right_fit)
    result = draw_lane(img, binary_warped, Left.best_fit, Right.best_fit, Minv)
    result = draw_text(result, radius, center_offset)
    
    return result

## Import Video Package for loading test Videos

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

### Applying pipeline on project video

In [ ]:
project_output = 'project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip1 = VideoFileClip("project_video.mp4").subclip(0,5)
# clip1 = VideoFileClip("project_video.mp4")
project_clip = clip1.fl_image(process_frame) #NOTE: this function expects color images!!
%time project_clip.write_videofile(project_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(project_output))

### Applying pipeline on Challenge Video

In [ ]:
challenge_output = 'challenge_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip2 = VideoFileClip("challenge_video.mp4").subclip(0,5)
# clip2 = VideoFileClip("challenge_video.mp4")
challenge_clip = clip2.fl_image(process_frame) #NOTE: this function expects color images!!
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))

### Applying pipeline on Harder Challenge Video

In [ ]:
harder_challenge_output = 'harder_challenge_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
clip3 = VideoFileClip("harder_challenge_video.mp4").subclip(0,5)
# clip3 = VideoFileClip("harder_challenge_video.mp4")
harder_challenge_clip = clip3.fl_image(process_frame) #NOTE: this function expects color images!!
%time harder_challenge_clip.write_videofile(harder_challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(harder_challenge_output))